TODO
- add code to drop undesirable variables for goalkeepers

In [1]:
# number of latest gameweek
gameweek = 34

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt
import pickle
import json
import requests

import shap
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from scipy.stats import poisson
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# allow more data columns/rows to be shown than by default
pd.set_option('display.max_columns', 501)
pd.set_option('display.max_rows', 501)

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)
#display(player_name_dict)

In [4]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
display(fpl_df.head(5))

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,84450,0,0,-2,2,1,3,3.4,3.9,13,Granit,3.4,3,False,NaN,2023-04-22T08:30:07.774962Z,48,84450.jpg,4.0,Xhaka,2.5,False,NaN,a,Arsenal,3,132,895886,7142,811916,2004,0.7,27.5,Xhaka,2707,5,8,11,33,0,0,0,4,0,0,13,562,621.4,635.7,492.0,175.0,32,3.67,3.67,7.34,33.65,49,19,28,22,56,35,35,24,NaN,NaN,2.0,NaN,NaN,NaN,0.12,0.0,0.12,0.24,1.12,1.10,266,158,73,36,56,23,120,40,1.06,0.37,2022-23,Granit Xhaka,MID,34,4.8,285182.650,5138,Chelsea,2023-05-02T19:00:00Z,1.0
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,0.0,0,Mohamed,0.0,4,False,Knee injury - Unknown return date,2023-01-26T10:00:07.778526Z,41,153256.jpg,1.2,Elneny,0.6,False,NaN,i,Arsenal,3,6,200196,159,277688,370,0.0,1.5,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,501,226,454,220,585,231,500,226,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.03,0.03,1.05,1.62,626,326,510,185,439,192,247,80,0.81,0.00,2022-23,Mohamed Elneny,MID,34,4.1,68443.836,-211,Chelsea,2023-05-02T19:00:00Z,1.0
2,NaN,NaN,156074,0,0,-3,3,0,2,2.0,2.5,1,Rob,2.0,5,False,NaN,NaN,42,156074.jpg,1.5,Holding,0.2,False,NaN,a,Arsenal,3,21,20821,886,23693,578,0.5,5.0,Holding,562,1,0,0,13,0,0,0,0,0,0,0,120,152.0,10.3,54.0,21.6,6,0.32,0.15,0.47,11.14,320,128,442,157,328,111,386,139,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.0,0.02,0.07,1.78,2.08,590,163,156,45,401,142,366,138,0.96,0.00,2022-23,Rob Holding,DEF,34,4.2,22814.612,308,Chelsea,2023-05-02T19:00:00Z,1.0
3,100.0,100.0,167199,0,0,-3,3,0,3,1.4,1.9,1,Thomas,1.4,6,False,NaN,2023-03-31T15:30:08.770925Z,47,167199.jpg,2.8,Partey,0.4,False,NaN,a,Arsenal,3,80,108371,406,139912,542,0.3,17.0,Partey,2260,3,0,10,25,0,0,0,4,0,0,6,431,492.0,404.3,232.0,113.0,26,2.59,1.97,4.56,30.37,102,42,71,58,146,87,93,65,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.0,0.08,0.18,1.21,1.00,310,186,221,108,181,73,277,91,1.04,0.40,2022-23,Thomas Partey,MID,34,4.7,45629.224,-136,Chelsea,2023-05-02T19:00:00Z,1.0
4,100.0,100.0,184029,0,0,1,-1,5,3,7.2,7.7,15,Martin,7.2,7,True,NaN,2023-03-09T22:30:08.763464Z,66,184029.jpg,5.9,Ødegaard,26.9,False,NaN,a,Arsenal,3,195,6080808,96161,4810219,23888,1.1,29.5,Ødegaard,2813,14,8,11,36,0,0,0,4,0,0,28,749,909.0,1028.2,861.0,280.1,33,9.12,7.23,16.35,34.78,10,3,7,5,16,7,8,6,3.0,NaN,1.0,NaN,NaN,NaN,0.29,0.0,0.23,0.52,1.11,1.15,52,27,6,4,10,4,12,7,1.06,0.35,2022-23,Martin degaard,MID,34,6.6,3068565.314,72273,Chelsea,2023-05-02T19:00:00Z,1.0


In [5]:
fpl_fixtures = pd.read_csv('../data/fpl_fixtures.csv', index_col=0)
display(fpl_fixtures.head(5))

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911,Crystal Palace,Arsenal
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74914,Fulham,Liverpool
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912,Bournemouth,Aston Villa
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915,Leeds United,Wolves
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,74917,Newcastle Utd,Nottingham Forest


In [6]:
team_stats = pd.read_csv('../data/team_stats.csv', index_col=0, usecols=['date','team','team_xG', 'team_xGA'])
team_stats.head(5)

,team,team_xG,team_xGA
date,,,
2017-08-11 00:00:00+00:00,Arsenal,2.3,1.3
2017-08-11 00:00:00+00:00,Leicester City,1.3,2.3
2017-08-12 00:00:00+00:00,Watford,2.1,3.1
2017-08-12 00:00:00+00:00,Liverpool,3.1,2.1
2017-08-12 00:00:00+00:00,Crystal Palace,1.1,1.7


In [7]:
xgb_models = []
for i in range(1,11):
    xgb = XGBRegressor()
    xgb.load_model(f'../models/xgb_regression_full_game{i}.json')
    xgb_models.append(xgb)
display(xgb_models[0])
display(xgb_models[9])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8254703688775018,
             enable_categorical=False, gamma=0.25226144515542404, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.06285391203893921, max_delta_step=0, max_depth=3,
             min_child_weight=1.8725484088145359, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.7993913128492731, tree_method='auto',
             validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5195376619139985,
             enable_categorical=False, gamma=0.35139253750058574, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.03261313301598562, max_delta_step=0, max_depth=3,
             min_child_weight=0.9469696895323456, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.804844605129602, tree_method='auto',
             validate_parameters=1, verbosity=None)

In [8]:
player_data = pd.read_csv('../data/data_for_projections.csv', index_col=0)
player_data['name'] = player_data['name'].map(player_name_dict).fillna(player_data['name'])
player_data.head(5)

,own_goals,penalties_missed,saves,goals_conceded,penalties_saved,was_home,assists,GW,season,bps,transfers_balance,kickoff_time,selected,red_cards,goals_scored,name,total_points,value,minutes,creativity,clean_sheets,opp_team_name,bonus,influence,position,yellow_cards,transfers_in,ict_index,transfers_out,threat,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,General_xAG,Pass Types_TI,Dribbles_Mis,Dribbles_Dis,Passes_PrgP,Carries_PrgC,Take-Ons_Att,Take-Ons_Succ,General_PrgP,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Receiving_PrgR,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD,fbref_position_GK,fbref_position_LB,fbref_position_CB,fbref_position_RB,fbref_position_WB,fbref_position_DM,fbref_position_LM,fbref_position_CM,fbref_position_RM,fbref_position_AM,fbref_position_LW,fbref_position_RW,fbref_position_FW,team_xG,team_xGA,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG,opp_team_xGA,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0,xPoints,games_in_past_5_days_shifted_0,games_in_past_10_days_shifted_0,games_in_past_20_days_shifted_0,games_in_past_40_days_shifted_0,selected_shifted_0,transfers_balance_shifted_0,value_shifted_0,transfers_balance_%_shifted_0,transfers_balance_%_ewa5_shifted_0,transfers_balance_%_ewa10_shifted_0,value_change_1_shifted_0,value_change_2_shifted_0,value_change_5_shifted_0,goal_to_xg_ratio_ewa_40_shifted0,total_points_ewa_5_shifted_0,xPoints_ewa_5_shifted_0,assists_ewa_5_shifted_0,bonus_ewa_5_shifted_0,bps_ewa_5_shifted_0,clean_sheets_ewa_5_shifted_0,creativity_ewa_5_shifted_0,goals_conceded_ewa_5_shifted_0,goals_scored_ewa_5_shifted_0,ict_index_ewa_5_shifted_0,influence_ewa_5_shifted_0,minutes_ewa_5_shifted_0,own_goals_ewa_5_shifted_0,penalties_missed_ewa_5_shifted_0,pen

In [9]:
keep = [
        'was_home', 'fpl_position_GK', 'fpl_position_DEF',
       'fpl_position_MID', 'fpl_position_FWD', 'team_xG_ewa_5_shifted_0',
       'team_xG_ewa_10_shifted_0', 'team_xG_ewa_20_shifted_0',
       'team_xG_ewa_40_shifted_0', 'team_xGA_ewa_5_shifted_0',
       'team_xGA_ewa_10_shifted_0', 'team_xGA_ewa_20_shifted_0',
       'team_xGA_ewa_40_shifted_0', 'opp_team_xG_ewa_5_shifted_0',
       'opp_team_xG_ewa_10_shifted_0', 'opp_team_xG_ewa_20_shifted_0',
       'opp_team_xG_ewa_40_shifted_0', 'opp_team_xGA_ewa_5_shifted_0',
       'opp_team_xGA_ewa_10_shifted_0', 'opp_team_xGA_ewa_20_shifted_0',
       'opp_team_xGA_ewa_40_shifted_0', 'games_in_past_5_days_shifted_0',
       'games_in_past_10_days_shifted_0',
       'games_in_past_20_days_shifted_0',
       'games_in_past_40_days_shifted_0',
       'transfers_balance_%_shifted_0',
       'transfers_balance_%_ewa5_shifted_0',
       'transfers_balance_%_ewa10_shifted_0',
       'value_change_1_shifted_0', 'value_change_2_shifted_0',
       'value_change_5_shifted_0', 'goal_to_xg_ratio_ewa_40_shifted0',
       'total_points_ewa_5_shifted_0', 'xPoints_ewa_5_shifted_0',
       'assists_ewa_5_shifted_0', 'bonus_ewa_5_shifted_0',
       'bps_ewa_5_shifted_0', 'clean_sheets_ewa_5_shifted_0',
       'creativity_ewa_5_shifted_0', 'goals_conceded_ewa_5_shifted_0',
       'goals_scored_ewa_5_shifted_0', 'ict_index_ewa_5_shifted_0',
       'influence_ewa_5_shifted_0', 'minutes_ewa_5_shifted_0',
       'own_goals_ewa_5_shifted_0', 'penalties_missed_ewa_5_shifted_0',
       'penalties_saved_ewa_5_shifted_0', 'saves_ewa_5_shifted_0',
       'threat_ewa_5_shifted_0', 'yellow_cards_ewa_5_shifted_0',
       'Performance_Sh_ewa_5_shifted_0',
       'Performance_SoT_ewa_5_shifted_0',
       'Performance_Touches_ewa_5_shifted_0',
       'Performance_Press_ewa_5_shifted_0',
       'Performance_Tkl_ewa_5_shifted_0',
       'Performance_Int_ewa_5_shifted_0',
       'Performance_Blocks_ewa_5_shifted_0',
       'Expected_xG_ewa_5_shifted_0', 'Expected_npxG_ewa_5_shifted_0',
       'Expected_xA_ewa_5_shifted_0', 'SCA_SCA_ewa_5_shifted_0',
       'Passes_Cmp_ewa_5_shifted_0', 'Passes_Cmp%_ewa_5_shifted_0',
       'Passes_Prog_ewa_5_shifted_0', 'Carries_Carries_ewa_5_shifted_0',
       'Carries_Prog_ewa_5_shifted_0', 'Dribbles_Succ_ewa_5_shifted_0',
       'Dribbles_Att_ewa_5_shifted_0', 'General_KP_ewa_5_shifted_0',
       'Touches_Att 3rd_ewa_5_shifted_0',
       'Touches_Att Pen_ewa_5_shifted_0',
       'Performance_Recov_ewa_5_shifted_0',
       'Aerial Duels_Won_ewa_5_shifted_0',
       'Aerial Duels_Won%_ewa_5_shifted_0',
       'fbref_position_GK_ewa5_shifted_0',
       'fbref_position_LB_ewa5_shifted_0',
       'fbref_position_CB_ewa5_shifted_0',
       'fbref_position_RB_ewa5_shifted_0',
       'fbref_position_WB_ewa5_shifted_0',
       'fbref_position_DM_ewa5_shifted_0',
       'fbref_position_LM_ewa5_shifted_0',
       'fbref_position_CM_ewa5_shifted_0',
       'fbref_position_RM_ewa5_shifted_0',
       'fbref_position_AM_ewa5_shifted_0',
       'fbref_position_LW_ewa5_shifted_0',
       'fbref_position_RW_ewa5_shifted_0',
       'fbref_position_FW_ewa5_shifted_0',
       'total_points_ewa_10_shifted_0', 'xPoints_ewa_10_shifted_0',
       'assists_ewa_10_shifted_0', 'bonus_ewa_10_shifted_0',
       'bps_ewa_10_shifted_0', 'clean_sheets_ewa_10_shifted_0',
       'creativity_ewa_10_shifted_0', 'goals_conceded_ewa_10_shifted_0',
       'goals_scored_ewa_10_shifted_0', 'ict_index_ewa_10_shifted_0',
       'influence_ewa_10_shifted_0', 'minutes_ewa_10_shifted_0',
       'own_goals_ewa_10_shifted_0',
       'penalties_missed_ewa_10_shifted_0',
       'penalties_saved_ewa_10_shifted_0', 'saves_ewa_10_shifted_0',
       'threat_ewa_10_shifted_0', 'yellow_cards_ewa_10_shifted_0',
       'Performance_Sh_ewa_10_shifted_0',
       'Performance_SoT_ewa_10_shifted_0',
       'Performance_Touches_ewa_10_shifted_0',
       'Performance_Press_ewa_10_shifted_0',
       'Performance_Tkl_ewa_10_shifted_0',
       'Performance_Int_ewa_10_shifted_0',
       'Performance_Blocks_ewa_10_shifted_0',
       'Expected_xG_ewa_10_shifted_0', 'Expected_npxG_ewa_10_shifted_0',
       'Expected_xA_ewa_10_shifted_0', 'SCA_SCA_ewa_10_shifted_0',
       'Passes_Cmp_ewa_10_shifted_0', 'Passes_Cmp%_ewa_10_shifted_0',
       'Passes_Prog_ewa_10_shifted_0',
       'Carries_Carries_ewa_10_shifted_0',
       'Carries_Prog_ewa_10_shifted_0',
       'Dribbles_Succ_ewa_10_shifted_0',
       'Dribbles_Att_ewa_10_shifted_0', 'General_KP_ewa_10_shifted_0',
       'Touches_Att 3rd_ewa_10_shifted_0',
       'Touches_Att Pen_ewa_10_shifted_0',
       'Performance_Recov_ewa_10_shifted_0',
       'Aerial Duels_Won_ewa_10_shifted_0',
       'Aerial Duels_Won%_ewa_10_shifted_0',
       'fbref_position_GK_ewa10_shifted_0',
       'fbref_position_LB_ewa10_shifted_0',
       'fbref_position_CB_ewa10_shifted_0',
       'fbref_position_RB_ewa10_shifted_0',
       'fbref_position_WB_ewa10_shifted_0',
       'fbref_position_DM_ewa10_shifted_0',
       'fbref_position_LM_ewa10_shifted_0',
       'fbref_position_CM_ewa10_shifted_0',
       'fbref_position_RM_ewa10_shifted_0',
       'fbref_position_AM_ewa10_shifted_0',
       'fbref_position_LW_ewa10_shifted_0',
       'fbref_position_RW_ewa10_shifted_0',
       'fbref_position_FW_ewa10_shifted_0',
       'total_points_ewa_20_shifted_0', 'xPoints_ewa_20_shifted_0',
       'assists_ewa_20_shifted_0', 'bonus_ewa_20_shifted_0',
       'bps_ewa_20_shifted_0', 'clean_sheets_ewa_20_shifted_0',
       'creativity_ewa_20_shifted_0', 'goals_conceded_ewa_20_shifted_0',
       'goals_scored_ewa_20_shifted_0', 'ict_index_ewa_20_shifted_0',
       'influence_ewa_20_shifted_0', 'minutes_ewa_20_shifted_0',
       'own_goals_ewa_20_shifted_0',
       'penalties_missed_ewa_20_shifted_0',
       'penalties_saved_ewa_20_shifted_0', 'saves_ewa_20_shifted_0',
       'threat_ewa_20_shifted_0', 'yellow_cards_ewa_20_shifted_0',
       'Performance_Sh_ewa_20_shifted_0',
       'Performance_SoT_ewa_20_shifted_0',
       'Performance_Touches_ewa_20_shifted_0',
       'Performance_Press_ewa_20_shifted_0',
       'Performance_Tkl_ewa_20_shifted_0',
       'Performance_Int_ewa_20_shifted_0',
       'Performance_Blocks_ewa_20_shifted_0',
       'Expected_xG_ewa_20_shifted_0', 'Expected_npxG_ewa_20_shifted_0',
       'Expected_xA_ewa_20_shifted_0', 'SCA_SCA_ewa_20_shifted_0',
       'Passes_Cmp_ewa_20_shifted_0', 'Passes_Cmp%_ewa_20_shifted_0',
       'Passes_Prog_ewa_20_shifted_0',
       'Carries_Carries_ewa_20_shifted_0',
       'Carries_Prog_ewa_20_shifted_0',
       'Dribbles_Succ_ewa_20_shifted_0',
       'Dribbles_Att_ewa_20_shifted_0', 'General_KP_ewa_20_shifted_0',
       'Touches_Att 3rd_ewa_20_shifted_0',
       'Touches_Att Pen_ewa_20_shifted_0',
       'Performance_Recov_ewa_20_shifted_0',
       'Aerial Duels_Won_ewa_20_shifted_0',
       'Aerial Duels_Won%_ewa_20_shifted_0',
       'fbref_position_GK_ewa20_shifted_0',
       'fbref_position_LB_ewa20_shifted_0',
       'fbref_position_CB_ewa20_shifted_0',
       'fbref_position_RB_ewa20_shifted_0',
       'fbref_position_WB_ewa20_shifted_0',
       'fbref_position_DM_ewa20_shifted_0',
       'fbref_position_LM_ewa20_shifted_0',
       'fbref_position_CM_ewa20_shifted_0',
       'fbref_position_RM_ewa20_shifted_0',
       'fbref_position_AM_ewa20_shifted_0',
       'fbref_position_LW_ewa20_shifted_0',
       'fbref_position_RW_ewa20_shifted_0',
       'fbref_position_FW_ewa20_shifted_0',
       'total_points_ewa_40_shifted_0', 'xPoints_ewa_40_shifted_0',
       'assists_ewa_40_shifted_0', 'bonus_ewa_40_shifted_0',
       'bps_ewa_40_shifted_0', 'clean_sheets_ewa_40_shifted_0',
       'creativity_ewa_40_shifted_0', 'goals_conceded_ewa_40_shifted_0',
       'goals_scored_ewa_40_shifted_0', 'ict_index_ewa_40_shifted_0',
       'influence_ewa_40_shifted_0', 'minutes_ewa_40_shifted_0',
       'own_goals_ewa_40_shifted_0',
       'penalties_missed_ewa_40_shifted_0',
       'penalties_saved_ewa_40_shifted_0', 'saves_ewa_40_shifted_0',
       'threat_ewa_40_shifted_0', 'yellow_cards_ewa_40_shifted_0',
       'Performance_Sh_ewa_40_shifted_0',
       'Performance_SoT_ewa_40_shifted_0',
       'Performance_Touches_ewa_40_shifted_0',
       'Performance_Press_ewa_40_shifted_0',
       'Performance_Tkl_ewa_40_shifted_0',
       'Performance_Int_ewa_40_shifted_0',
       'Performance_Blocks_ewa_40_shifted_0',
       'Expected_xG_ewa_40_shifted_0', 'Expected_npxG_ewa_40_shifted_0',
       'Expected_xA_ewa_40_shifted_0', 'SCA_SCA_ewa_40_shifted_0',
       'Passes_Cmp_ewa_40_shifted_0', 'Passes_Cmp%_ewa_40_shifted_0',
       'Passes_Prog_ewa_40_shifted_0',
       'Carries_Carries_ewa_40_shifted_0',
       'Carries_Prog_ewa_40_shifted_0',
       'Dribbles_Succ_ewa_40_shifted_0',
       'Dribbles_Att_ewa_40_shifted_0', 'General_KP_ewa_40_shifted_0',
       'Touches_Att 3rd_ewa_40_shifted_0',
       'Touches_Att Pen_ewa_40_shifted_0',
       'Performance_Recov_ewa_40_shifted_0',
       'Aerial Duels_Won_ewa_40_shifted_0',
       'Aerial Duels_Won%_ewa_40_shifted_0',
       'fbref_position_GK_ewa40_shifted_0',
       'fbref_position_LB_ewa40_shifted_0',
       'fbref_position_CB_ewa40_shifted_0',
       'fbref_position_RB_ewa40_shifted_0',
       'fbref_position_WB_ewa40_shifted_0',
       'fbref_position_DM_ewa40_shifted_0',
       'fbref_position_LM_ewa40_shifted_0',
       'fbref_position_CM_ewa40_shifted_0',
       'fbref_position_RM_ewa40_shifted_0',
       'fbref_position_AM_ewa40_shifted_0',
       'fbref_position_LW_ewa40_shifted_0',
       'fbref_position_RW_ewa40_shifted_0',
       'fbref_position_FW_ewa40_shifted_0'
]
print(len(keep))

252


In [10]:
team_features = [
        'team_xG_ewa_5_shifted_0',
       'team_xG_ewa_10_shifted_0', 'team_xG_ewa_20_shifted_0',
       'team_xG_ewa_40_shifted_0', 'team_xGA_ewa_5_shifted_0',
       'team_xGA_ewa_10_shifted_0', 'team_xGA_ewa_20_shifted_0',
       'team_xGA_ewa_40_shifted_0'
]
opponent_features = [
    'opp_team_xG_ewa_5_shifted_0',
       'opp_team_xG_ewa_10_shifted_0', 'opp_team_xG_ewa_20_shifted_0',
       'opp_team_xG_ewa_40_shifted_0', 'opp_team_xGA_ewa_5_shifted_0',
       'opp_team_xGA_ewa_10_shifted_0', 'opp_team_xGA_ewa_20_shifted_0',
       'opp_team_xGA_ewa_40_shifted_0'
]

In [11]:
teams = fpl_fixtures['home_team'].unique()
teams

array(['Crystal Palace', 'Fulham', 'Bournemouth', 'Leeds United',
       'Newcastle Utd', 'Tottenham', 'Everton', 'Leicester City',
       'Manchester Utd', 'West Ham', 'Aston Villa', 'Arsenal', 'Brighton',
       'Manchester City', 'Southampton', 'Wolves', 'Brentford',
       'Nottingham Forest', 'Chelsea', 'Liverpool'], dtype=object)

In [12]:
team_stats_for_predictions = pd.DataFrame(columns=[team_features + opponent_features])
for team in teams:
    team_stats_for_predictions.loc[team, 'team_xG_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xG_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/40).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'team_xGA_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/40).mean().values[-1]
    
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xG_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xG'].ewm(alpha=1/40).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_5_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/5).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_10_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/10).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_20_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/20).mean().values[-1]
    team_stats_for_predictions.loc[team, 'opp_team_xGA_ewa_40_shifted_0'] = \
                team_stats.loc[team_stats['team']==team, 'team_xGA'].ewm(alpha=1/40).mean().values[-1]

aux_columns = ['opp_team_xG_ewa_5_shifted_0', 'opp_team_xG_ewa_10_shifted_0','opp_team_xG_ewa_20_shifted_0',
               'opp_team_xG_ewa_40_shifted_0']
team_stats_for_predictions.loc['Nottingham Forest', aux_columns] = team_stats_for_predictions.loc['Fulham', aux_columns].copy()
aux_columns = ['opp_team_xGA_ewa_5_shifted_0', 'opp_team_xGA_ewa_10_shifted_0','opp_team_xGA_ewa_20_shifted_0',
               'opp_team_xGA_ewa_40_shifted_0']
team_stats_for_predictions.loc['Nottingham Forest', aux_columns] = team_stats_for_predictions.loc['Bournemouth', aux_columns].copy()

team_stats_for_predictions.head(5)

,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0
Crystal Palace,1.296219,1.15358,1.085725,1.062358,1.148677,1.192221,1.227279,1.255083,1.296219,1.15358,1.085725,1.062358,1.148677,1.192221,1.227279,1.255083
Fulham,0.841943,0.976914,1.075514,1.106338,1.646901,1.607588,1.570057,1.551593,0.841943,0.976914,1.075514,1.106338,1.646901,1.607588,1.570057,1.551593
Bournemouth,1.494599,1.35854,1.21944,1.168939,1.660186,1.727832,1.70183,1.645074,1.494599,1.35854,1.21944,1.168939,1.660186,1.727832,1.70183,1.645074
Leeds United,1.270227,1.220537,1.228086,1.257184,2.078643,1.946275,1.83083,1.777806,1.270227,1.220537,1.228086,1.257184,2.078643,1.946275,1.83083,1.777806
Newcastle Utd,2.62515,2.211094,1.864464,1.555349,1.13622,1.110727,1.115233,1.19631,2.62515,2.211094,1.864464,1.555349,1.13622,1.110727,1.115233,1.19631


In [13]:
player_names = fpl_df.loc[fpl_df['minutes']>0, 'name'].unique()
players_not_projected = []
for player_name in player_names:   
    if player_data[player_data['name']==player_name].shape[0]==0:
        players_not_projected.append(player_name)
display(players_not_projected)
display(len(players_not_projected))

['Fabio Ferreira Vieira',
 'Marcus Oliveira Alencar',
 'Jorge Luiz Frello Filho',
 'Diego Carlos Santos Silva',
 'Alexandre Moreno Lopera',
 'Norberto Murara Neto',
 'Hamed Traore',
 'Sergi Canos Tenes',
 'Mads Roerslev Rasmussen',
 'Mathias Jorgensen',
 'Jeremy Sarmiento Morante',
 'Julio Enciso',
 'Yasin Ayari',
 'Omari Hutchinson',
 'David Datro Fofana',
 'Cedric Alves Soares',
 'Carlos Vinicius Alves Morais',
 'Ricardo Barbosa Pereira',
 'Victor Kristiansen',
 'Mateus Cardoso Lemos Martins',
 'Junior Firpo Adames',
 'Mateo Joseph Fernandez',
 'Fabio Henrique Tavares',
 'Stefan Ortega Moreno',
 'Frederico Rodrigues de Paula Santos',
 'Facundo Pellistri Rebollo',
 'Javier Manquillo Gaitan',
 'Carlos Ribeiro Dias',
 'Alex Mighten',
 'Renan Augusto Lodi dos Santos',
 'Gustavo Henrique Furtado Scarpa',
 'Danilo dos Santos de Oliveira',
 'Felipe Augusto de Almeida Monteiro',
 'Lyanco Silveira Neves Vojnovic',
 'Armel Bella-Kotchap',
 'Juan Larios Lopez',
 'Emerson Leite de Souza Junior',

43

In [14]:
%%time
# run xP projections for each player
first_projected_gameweek=gameweek+1
horizon=10
player_names = fpl_df['name'].unique()
all_predictions = []
prediction_data = pd.DataFrame()
players_not_projected = []
for player_name in player_names:   
    if player_data[player_data['name']==player_name].shape[0]>0:
        team = fpl_df.loc[fpl_df['name']==player_name, 'team'].values[0]
        #price = fpl_df.loc[fpl_df['name']==player_name, 'now_cost'].values[0] / 10.0
        aux = player_data[player_data['name']==player_name].tail(1)
        for count, week in enumerate(np.arange(first_projected_gameweek, first_projected_gameweek+horizon)):
            home_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['home_team']==team)]
            away_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['away_team']==team)]
            for ix in home_games.index:
                aux['was_home'] = 1                
                aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
                opp_team = fpl_fixtures.loc[ix, 'away_team']
                aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()                
                prediction_data = pd.concat([prediction_data, aux])
                xP = xgb_models[count].predict(aux[keep])[0]
                all_predictions.append([player_name, team, opp_team, 1, week, xP])
            for ix in away_games.index:
                aux['was_home'] = 0
                aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
                opp_team = fpl_fixtures.loc[ix, 'home_team']
                aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
                prediction_data = pd.concat([prediction_data, aux])
                xP = xgb_models[count].predict(aux[keep])[0]
                all_predictions.append([player_name, team, opp_team, 0, week, xP])
    else:
        players_not_projected.append(player_name)

my_columns = ['name', 'team', 'opp_team', 'was_home', 'gameweek', 'xP']
predictions = pd.DataFrame(all_predictions, columns=my_columns)
prediction_data = prediction_data.reset_index(drop=True)
display(prediction_data.head(5))

,own_goals,penalties_missed,saves,goals_conceded,penalties_saved,was_home,assists,GW,season,bps,transfers_balance,kickoff_time,selected,red_cards,goals_scored,name,total_points,value,minutes,creativity,clean_sheets,opp_team_name,bonus,influence,position,yellow_cards,transfers_in,ict_index,transfers_out,threat,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,General_xAG,Pass Types_TI,Dribbles_Mis,Dribbles_Dis,Passes_PrgP,Carries_PrgC,Take-Ons_Att,Take-Ons_Succ,General_PrgP,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Receiving_PrgR,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD,fbref_position_GK,fbref_position_LB,fbref_position_CB,fbref_position_RB,fbref_position_WB,fbref_position_DM,fbref_position_LM,fbref_position_CM,fbref_position_RM,fbref_position_AM,fbref_position_LW,fbref_position_RW,fbref_position_FW,team_xG,team_xGA,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG,opp_team_xGA,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0,xPoints,games_in_past_5_days_shifted_0,games_in_past_10_days_shifted_0,games_in_past_20_days_shifted_0,games_in_past_40_days_shifted_0,selected_shifted_0,transfers_balance_shifted_0,value_shifted_0,transfers_balance_%_shifted_0,transfers_balance_%_ewa5_shifted_0,transfers_balance_%_ewa10_shifted_0,value_change_1_shifted_0,value_change_2_shifted_0,value_change_5_shifted_0,goal_to_xg_ratio_ewa_40_shifted0,total_points_ewa_5_shifted_0,xPoints_ewa_5_shifted_0,assists_ewa_5_shifted_0,bonus_ewa_5_shifted_0,bps_ewa_5_shifted_0,clean_sheets_ewa_5_shifted_0,creativity_ewa_5_shifted_0,goals_conceded_ewa_5_shifted_0,goals_scored_ewa_5_shifted_0,ict_index_ewa_5_shifted_0,influence_ewa_5_shifted_0,minutes_ewa_5_shifted_0,own_goals_ewa_5_shifted_0,penalties_missed_ewa_5_shifted_0,pen

CPU times: user 2min 47s, sys: 33.8 s, total: 3min 20s
Wall time: 28.5 s


In [15]:
predictions.head(5)

,name,team,opp_team,was_home,gameweek,xP
0,Granit Xhaka,Arsenal,Newcastle Utd,0,35,3.069497
1,Granit Xhaka,Arsenal,Brighton,1,36,2.910045
2,Granit Xhaka,Arsenal,Nottingham Forest,0,37,4.172058
3,Granit Xhaka,Arsenal,Wolves,1,38,3.443727
4,Mohamed Elneny,Arsenal,Newcastle Utd,0,35,2.025140


In [16]:
aux = predictions[predictions['name'].str.contains('Haaland')].copy()
aux['xP_cumulative'] = aux['xP'].cumsum()
display(aux)
ppg = aux['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')

,name,team,opp_team,was_home,gameweek,xP,xP_cumulative
1552,Erling Haaland,Manchester City,Leeds United,1,35,5.882340,5.882340
1553,Erling Haaland,Manchester City,Everton,0,36,3.644258,9.526599
1554,Erling Haaland,Manchester City,Chelsea,1,37,6.681406,16.208004
1555,Erling Haaland,Manchester City,Brighton,0,37,5.049343,21.257347
1556,Erling Haaland,Manchester City,Brentford,0,38,5.508060,26.765408


Points per game: 5.349999904632568


In [17]:
predictions.to_csv(f'../point_projections/xP_projections_week{gameweek}.csv')

# Collective predictions

Here we use all the 10 models to make collective predictions (mean prediction)

In [18]:
%%time
# run xP projections for each player by using each model and taking the average
my_predictions = []
my_std = []
for ix in prediction_data.index:
    model_predictions = []
    for model_ix in range(10):
        model_predictions.append(xgb_models[model_ix].predict(prediction_data.loc[ix:ix, keep])[0])
    my_predictions.append(np.mean(model_predictions))
    my_std.append(np.std(model_predictions))

CPU times: user 8min 25s, sys: 1min 37s, total: 10min 2s
Wall time: 1min 25s


In [19]:
collective_predictions = predictions.copy()
collective_predictions['xP'] = my_predictions
collective_predictions['std'] = my_std
collective_predictions.head(5)

,name,team,opp_team,was_home,gameweek,xP,std
0,Granit Xhaka,Arsenal,Newcastle Utd,0,35,2.721090,0.150307
1,Granit Xhaka,Arsenal,Brighton,1,36,2.910805,0.127589
2,Granit Xhaka,Arsenal,Nottingham Forest,0,37,3.551062,0.266156
3,Granit Xhaka,Arsenal,Wolves,1,38,3.740756,0.224557
4,Mohamed Elneny,Arsenal,Newcastle Utd,0,35,2.128134,0.146237


In [20]:
aux = collective_predictions[collective_predictions['name']=='Erling Haaland']
display(aux)
ppg = aux['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')

,name,team,opp_team,was_home,gameweek,xP,std
1552,Erling Haaland,Manchester City,Leeds United,1,35,6.499094,1.365229
1553,Erling Haaland,Manchester City,Everton,0,36,5.737916,1.171730
1554,Erling Haaland,Manchester City,Chelsea,1,37,4.690942,0.959745
1555,Erling Haaland,Manchester City,Brighton,0,37,3.751135,0.868083
1556,Erling Haaland,Manchester City,Brentford,0,38,4.310575,0.821085


Points per game: 5.0


In [21]:
collective_predictions.to_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv')

# Additional/alternative projections

Eearly season, the model has very little data on some new player, e.g. Haaland, so I'll supplement the data by giving him Kane's statistics for history and then updating those with new Haaland data as it comes

In [ ]:
def alternative_projections(original_player_name, mock_player_name, original_team):
    
    player_name = mock_player_name
    real_player_name = original_player_name
    team = original_team
    aux1 = player_data.loc[player_data['name']==player_name, keep].tail(1)
    aux2 = player_data.loc[player_data['name']==real_player_name, columns_for_ewa+keep]
    
    ewa_intervals = [5,10,20,40]
    for interval in ewa_intervals:
        for col in columns_for_ewa:
            aux2[f'{col}_ewa_{interval}_shifted_0'] = (1-1/interval)*aux1[f'{col}_ewa_{interval}_shifted_0'].values[0] +\
                                                        (1/interval)*aux2[col].values[0]
            
    # mock predictions 
    first_projected_gameweek=gameweek+1
    horizon=10

    aux=aux2.copy()
    all_predictions = []
    aux_prediction_data = pd.DataFrame()
    for count, week in enumerate(np.arange(first_projected_gameweek, first_projected_gameweek+horizon)):
        home_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['home_team']==team)]
        away_games = fpl_fixtures[(fpl_fixtures['event']==week) & (fpl_fixtures['away_team']==team)]
        for ix in home_games.index:
            aux['was_home'] = 1
            aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
            opp_team = fpl_fixtures.loc[ix, 'away_team']
            aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
            aux_prediction_data = pd.concat([aux_prediction_data,aux])
            xP = xgb_models[count].predict(aux[keep])[0]
            all_predictions.append([real_player_name, team, opp_team, 1, week, xP])
        for ix in away_games.index:
            aux['was_home'] = 0
            aux[team_features] = team_stats_for_predictions.loc[team, team_features].copy()
            opp_team = fpl_fixtures.loc[ix, 'home_team']
            aux[opponent_features] = team_stats_for_predictions.loc[opp_team, opponent_features].copy()
            aux_prediction_data = pd.concat([aux_prediction_data,aux])
            xP = xgb_models[count].predict(aux[keep])[0]
            all_predictions.append([real_player_name, team, opp_team, 0, week, xP])

    all_predictions = pd.DataFrame(all_predictions, columns=my_columns)
    
    return all_predictions

In [ ]:
def add_alternative_projections(real_player_name, predictions, aux_prediction_data):
    
    predictions = predictions.drop(predictions[predictions['name']==real_player_name].index)
    predictions = pd.concat([predictions, all_predictions])
    predictions = predictions.reset_index(drop=True)
    
    return predictions 

In [ ]:
columns_for_ewa = ['total_points', 'xPoints', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
                   'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
                   'penalties_saved', 'saves', 'threat', 'yellow_cards', 
                'Performance_Sh', 'Performance_SoT', 'Performance_Touches', 'Performance_Press', 'Performance_Tkl',
                  'Performance_Int', 'Performance_Blocks', 'Expected_xG', 'Expected_npxG', 'Expected_xA',
                  'SCA_SCA', 'Passes_Cmp', 'Passes_Cmp%', 'Passes_Prog', 'Carries_Carries', 'Carries_Prog',
                  'Dribbles_Succ', 'Dribbles_Att', 'General_KP', 'Touches_Att 3rd', 'Touches_Att Pen', 
                   'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Won%']

In [ ]:
original_player_name = 'Erling Haaland'
mock_player_name = 'Harry Kane'
original_team = 'Manchester City'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
ppg = aux_prediction_data['xP'].mean()
print(f'Points per game: {np.round(ppg,2)}')
display(aux_prediction_data)

In [ ]:
original_player_name = 'Ivan Perisic'
mock_player_name = 'Andrew Robertson'
original_team = 'Tottenham'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
aux_prediction_data

In [ ]:
original_player_name = 'Darwin Nunez'
mock_player_name = 'Harry Kane'
original_team = 'Liverpool'
aux_prediction_data = alternative_projections(original_player_name, mock_player_name, original_team)
aux_prediction_data

In [ ]:
#predictions.to_csv(f'../data/xP_projections_week{gameweek}.csv')

# Shap-values for individual projections

In [ ]:
player_name = 'Erling Haaland'
ixs = predictions.loc[predictions['name']==player_name].index
predictions.loc[predictions['name']==player_name]

In [ ]:
explainer = shap.Explainer(xgb_models[0], prediction_data[keep])
shap_values = explainer(prediction_data[keep])

In [ ]:
shap.plots.waterfall(shap_values[ixs[0]], max_display=20)

In [ ]:
prediction_data[keep].loc[ixs[0]:ixs[0]]['bps_ewa_5_shifted_0']